# **Generation**

In [1]:
docs_feed = 5
parent_chunk_size = 500

In [2]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma langchain_groq ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 41.7 MB/s et

In [3]:
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain_core.output_parsers import StrOutputParser
import re
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.storage import InMemoryStore
from operator import itemgetter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from pydantic import BaseModel, Field
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from google.colab import userdata
from langchain import PromptTemplate
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import os
import json
from google.colab import files
import time
from langchain_groq import ChatGroq
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import CrossEncoder
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from ragas import SingleTurnSample
from ragas.metrics import ResponseRelevancy, LLMContextRecall
from ragas.llms import LangchainLLMWrapper
import math
import numpy as np

<br/>
<br/>
<br/>

## **User Action Required**

1. Run the code below to create the ```data``` folder

2. Choose to upload the following files
- ```iceland_articles.csv```
- ```finland_articles.csv```
- ```sweden_articles.csv```

In [4]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

In [5]:
uploaded_files = files.upload()

Saving finland_articles.csv to finland_articles.csv
Saving iceland_articles.csv to iceland_articles.csv
Saving sweden_articles.csv to sweden_articles.csv


In [6]:
for file_name in uploaded_files.keys():
    os.rename(file_name, os.path.join(data_folder, file_name))

Your folder structure should now look as such:

```
data
- iceland_articles.csv
- finland_articles.csv
- sweden_articles.csv
```

<br/>
<br/>
<br/>

## **Create document objects from the data and store in Docs**

In [7]:
article_names = ['finland_articles.csv', 'iceland_articles.csv', 'sweden_articles.csv']
article_fps = [os.path.join(data_folder, article_name) for article_name in article_names]

docs = []
for article_fp in article_fps:
  df = pd.read_csv(article_fp)
  for _, row in df.iterrows():
    #text = row['Title'] + " " + row['Content']
    text = row['Content']

    doc = Document(
        page_content=text,
        metadata={'country': row['Country'], 'source': row['Source'], 'link': row['Article Links']}
    )

    docs.append(doc)

<br/>
<br/>
<br/>

## **Initialise LLM**

In [8]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
# https://console.groq.com/settings/limits
llm = ChatGroq(model="mixtral-8x7b-32768")

<br/>
<br/>
<br/>

## **Define Query Decomposition Chain**

Decomposition is a query re-writing technique that focuses on decomposing a question into a set of subquestions.

This is applicable and effective for our use case as users planning a holiday tend to string together many requests in a single query. By breaking down a large/complex queries into sub-queries ('atomic' queries), the retriever can retrieve more relevant documents to each sub-query and therefore, support the LLM in answering the whole query better

In [9]:
# Prompt Decomposition template used by the LLM to help break a question into sub questions

# https://python.langchain.com/v0.1/docs/use_cases/query_analysis/techniques/decomposition/
# https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb
decomp_template = """You are an expert at breaking down travel-related questions into smaller, manageable sub-questions.
You have access to a collection of documents about various travel destinations, including information on activities, accommodations, local culture, and transportation.

Perform query decomposition. Given a user travel query, break it down into **distinct and independent** sub-questions that
you need to answer in order to comprehensively address the original query.

**Important:**
1. Ensure that the sub-questions are unambiguous, clear, and explicit. Avoid generic phrases such as "these activities" or "specific regions". Instead, directly refer to the activities, locations, or topics mentioned in the original query.
2. Ensure that the sub-questions are independent of each other and do not rely on answers from other sub-questions.
3. Cover diverse aspects of the query, ensuring that no sub-question overlaps or repeats similar information.
4. If there are location names, abbreviations, or specific terms you are unfamiliar with, do not attempt to rephrase them or make assumptions about their meaning.
5. **STRICTLY REFUSE to process non-travel-related queries.** If the query is unrelated to travel, explicitly respond with the following message and provide no other explanation:
   **"This query is not related to travel and cannot be broken down."**
6. Ensure your response adheres strictly to the format provided below. Do not add any additional text, explanations, or information outside of this structure.

Generate sub-questions related to: {question} \n

**Output Format (3 queries only):**
1. Sub-question 1
2. Sub-question 2
3. Sub-question 3

Here are a few examples:

---
Question: When is the best time to go Finland and what is there to do

Response:
1. What is the weather like in Finland during different times of the year?
2. What are the best and most suitable activities to do in Finland during each season?
3. Are there any cultural events or festivals in Finland that would influence the best time to visit?
---

---
Question: Help me plan a trip to Iceland, I am adventurous and like activities such as hiking

Response:
1. What are some popular hiking trails and locations in Iceland?
2. What is the best time of year to visit Iceland for hiking and outdoor activities?
3. What types of accommodations are available near the hiking areas in Iceland?
---

---
Question: How do I get around Iceland?

Response:
1. What are the different transportation options available for traveling within Iceland?
2. How can I travel between the international airport and Reykjavik, the capital city of Iceland?
3. Are there any notable road conditions or driving rules to be aware of when renting a car in Iceland?
---

---
Question: How old is Earth?

Response:
**This query is not related to travel and cannot be broken down.**
---

"""

decomp_prompt_template = PromptTemplate.from_template(decomp_template)

In [10]:
def clean_questions(raw_questions):
  questions = []
  raw_questions = raw_questions.split('\n')
  for qn in raw_questions:
    match = re.match(r"^\d+\.\s?", qn)
    if match:
      qn_number = match.group(0)
      qn_content = qn.replace(qn_number, '').strip()
      questions.append(qn_content)
  return questions

In [11]:
decomp_chain = ( decomp_prompt_template | llm | StrOutputParser() | clean_questions)

<br/>
<br/>
<br/>

## **Define Best Retriever from Retreiver Evaluation**

**Control Variables**

- Sub questions
- Retriever: FAISS with Index Flat L2
- Bi-Encoder/Embeddings model: all-mpnet-base-v2
- Text Split Method: Recursive Character Text Splitter
- Child Chunk Size: 250
- Child Chunk Overlap: 50
- Cross-Encoder/Re-ranking model: BAAI/bge-reranker-large
- Similarity Search type: MMR
- Number of docs to be fetched for MMR: 35
- Number of docs to be retrieved and reranked: 20

In [12]:
fetch_k = 35
top_k = 20
bi_encoder_embeddings_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=bi_encoder_embeddings_model_name)
cross_encoder_embedings_model_name = "BAAI/bge-reranker-large"
cross_encoder_model = HuggingFaceCrossEncoder(model_name=cross_encoder_embedings_model_name)
search_type = 'mmr'
child_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250, chunk_overlap=50, add_start_index=True
)

parent_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=parent_chunk_size, chunk_overlap=math.floor(0.2*parent_chunk_size), add_start_index=True
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [13]:
'''
Custom ParentDocumentRetriever object (ParentDocumentReranker) to do re-ranking of child chunks and return the parent chunks in the same order
'''
class ParentDocumentReranker(ParentDocumentRetriever):
    model_config = {"extra": "allow"}

    def __init__(self, cross_encoder_model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cross_encoder_model = cross_encoder_model

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Get documents relevant to a query.
        Args:
            query: String to find relevant documents for
            run_manager: The callbacks handler to use
        Returns:
            List of relevant documents
        """
        if self.search_kwargs['search_type'] == 'mmr':
            sub_docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            sub_docs = self.vectorstore.similarity_search(query, **self.search_kwargs)

        # Reranking logic here with sub_docs (child documents)
        sub_docs_rerank_vals = []
        for r in sub_docs:
          sub_docs_rerank_vals.append((r,self.cross_encoder_model.score((r.page_content,query))))
        sub_docs = [key for key, value in sorted(sub_docs_rerank_vals, key=lambda x: x[1], reverse=True)]

        # We do this to maintain the order of the ids that are returned
        ids = []
        for d in sub_docs:
            if self.id_key in d.metadata and d.metadata[self.id_key] not in ids:
                ids.append(d.metadata[self.id_key])
        docs = self.docstore.mget(ids)
        res = [d for d in docs if d is not None]
        if len(res) > self.search_kwargs['docs_feed']:
            return res[: self.search_kwargs['docs_feed']]
        return res

In [14]:
vector_store_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=vector_store_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

store = InMemoryStore()

custom_retriever = ParentDocumentReranker(
    cross_encoder_model=cross_encoder_model,
    vectorstore=faiss_vector_store,
    docstore=store,
    child_splitter=child_text_splitter,
    parent_splitter=parent_text_splitter,
    search_kwargs={"fetch_k": fetch_k, "k": top_k, "docs_feed": docs_feed, "search_type": search_type}
)

custom_retriever.add_documents(docs)

<br/>
<br/>
<br/>

## **Define Re-ordering Function**

Re-order re-ranked results such that extremas have the most relevent documents to mitigate "lost-in-the-middle" effect

In [15]:
def reorder_docs(res):
  res_reordered = [None] * len(res)
  start = 0
  end = len(res) - 1
  place_at_start = True

  for doc in res:
      if place_at_start:
          res_reordered[start] = doc
          start += 1
      else:
          res_reordered[end] = doc
          end -= 1

      place_at_start = not place_at_start
  return res_reordered

<br/>
<br/>
<br/>

## **Answer the Answerable Questions**

In [16]:
qna_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question. Each statement in your answer must be **entailable** by at least one of the provided citations, meaning the information in the statement must align directly and logically with the cited context.

### **IMPORTANT REQUIREMENTS**:
1. **Inline Numbered Citations**: For each statement in your answer, **you must include at least one inline numbered citation** (e.g., [1], [2]). Statements without sufficient support from the context should not be included in the answer.
2. **Entailment Verification**: Ensure that every statement (hypothesis) is directly entailed by the cited context (premise). Use only information explicitly stated or strongly implied by the citation. Do not include unsupported information or make up details.
3. **Refusal for Unsupported Queries**: If the retrieved context does not provide sufficient information to answer the question, explicitly respond with: **"I don't know based on the provided context."** Do not attempt to generate an answer.
4. **Sequential Citation Numbering**: Reset the numbering for citations to start from **1** for each response, and ensure the numbering increases sequentially.
5. **Single Paragraph**: Write your response as **a single paragraph**. Avoid using any new line characters in the response. All statements should flow naturally and seamlessly into one another.

At the bottom, provide the full citations corresponding to each number, but include **only the page content** of the Document Object (exclude metadata). Use citations exactly as provided in the context.

Respond in the following format:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1]: <Page Content 1> //Only the page content of the document
[2]: <Page Content 2>
[3]: <Page Content 3>
---

### **HOW TO APPROACH**:
- Before writing your answer, check if each statement is **entailable** by verifying that it is supported by one or more citations.
- If the context does not support a statement or the entire question, respond with: **"I don't know based on the provided context."**
- Avoid using unsupported statements, ambiguous language, or making up details.

Here are a few examples:

---
The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, Kjeragbolten, and Trolltunga [1, 2]. The Reinebringen hike, although not one of the highest peaks, offers an iconic view of the Reine fjord from its summit [1]. Preikestolen, Kjeragbolten, and Trolltunga are famous for their stunning fjord views and unique geological formations, such as a boulder stuck between a mountain crevasse and a tongue-shaped rock [2].

Citations:
[1]: "The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep."
[2]: "The most famous hikes in Norway include Preikestolen (a beautiful fjord), Kjeragbolten (with a famous boulder stuck between a mountain crevasse) as well as Trolltunga which resembles a tongue."
---

---
In Switzerland, you can embark on several scenic hikes [1]. One such hike is at Grindelwald, where at the summit, you will find a stunning lake, but it's only visible during the summer [1]. Other notable hikes include Zermatt, also known as the Matterhorn [1], and Lauterbrunnen [1]. For those seeking a challenging hike, the Matterhorn at Zermatt is a must-go [2]. This iconic peak is featured on the Toblerone chocolate and is best explored with a mountain guide due to the inherent dangers [2].

Citations:
[1]: "One of the most scenic hikes in Switzerland can be done at Grindelwald. At the summit of Grindelwald, a beautiful lake awaits you. However, you can only see this lake during summer time. Other notable hikes include Zermatt, i.e. the matterhorn and Lauterbrunnen."
[2]: "The matterhorn at zermatt is a must-go for hiking enthusiasts. It is the icon of the famous chocolate: Toblerone. However, it is recommended to hire a mountain guide to go with you as it can be very dangerous!"
---

---
In Iceland, you can participate in a variety of adventurous activities [1]. For instance, you can hike active volcanoes and explore a natural ice cave, offering unique geological experiences [1]. Driving in Iceland is also an amazing adventure, with open roads, majestic volcanoes, and towering mountains as your backdrop, and the possibility of encountering sheep and arctic foxes along the way [2]. Additionally, Iceland is known for its exceptional diving sites [3]. One of the most famous in the world, Silfra, is located in Iceland [3]. It is the only diving site where you can dive between two tectonic plates, and the water is so fresh that you can drink it, promising an unparalleled tasting experience [3].

Citations:
[1]: "Iceland is a must-go to place for adventurous people! You can hike active volcanoes, drive a jeep through the volcanic ash, explore a natural ice cave, see waterfalls. There are so many opportunities for an adventurer."
[2]: "Driving in Iceland is an amazing experience - open roads, majestic volcanos and towering mountains along the way, sheep and arctic foxes make it a great experience. All you need is an international driving license. And, please drive slowly during the winter!"
[3]: "One of the most famous diving sites in the world, Silfra, is located in Iceland! It is the only diving site in the world where you can dive between 2 tectonic plates. The water is also so fresh that you can drink from it, it is the best water that you will ever taste."
---

Question: {question}

Context: {context}

Helpful Answer:
"""

qna_template_shortened = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question. Each statement must be **entailable** by at least one of the provided citations.

### **REQUIREMENTS**:
1. **Inline Numbered Citations**: Include at least one inline numbered citation (e.g., [1], [2]) for each statement. Exclude unsupported statements.
2. **Entailment Only**: Ensure every statement is directly supported by the cited context. Do not include fabricated or ambiguous information.
3. **Single Paragraph**: Write your answer in one continuous paragraph with sequential citation numbering starting from **1**.
4. **Refuse Unsupported Queries**: If the context does not provide sufficient information, respond with: **"I don't know based on the provided context."**

At the bottom, provide the full citations for each number, using only the page content of the Document Object (exclude metadata).

**Format**:
---
Statement 1 [1]. Statement 2 [2, 3].

Citations:
[1]: <Page Content 1>
[2]: <Page Content 2>
[3]: <Page Content 3>
---

**Example**:
---
The best hikes in Norway include the Reinebringen hike in the Lofoten islands, Preikestolen, and Trolltunga [1, 2]. The Reinebringen hike offers an iconic view of the Reine fjord from its summit [1].

Citations:
[1]: "The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands."
[2]: "The most famous hikes in Norway include Preikestolen and Trolltunga."
---

Question: {question}

Context: {context}

Helpful Answer:
"""

qna_prompt_template = PromptTemplate.from_template(qna_template)

In [21]:
def get_answer(question,decomp_chain,retriever,answer_chain, sleep_time_s):
  final_response = {}
  print(f'Main Question: {question}')
  print('')
  sub_questions = decomp_chain.invoke({"question":question})
  for sub_question in sub_questions:
      print(f'Sub Question: {sub_question}')
      retrieved_docs = retriever.invoke(sub_question)
      retrieved_docs = reorder_docs(retrieved_docs)
      try:
        answer = answer_chain.invoke({"question": sub_question,"context": retrieved_docs})
      except Exception as e:
        print(f'Error: {e}')
        answer = None
      print(answer)
      print('')
      final_response[sub_question] = {
          "answer": answer,
      }
      time.sleep(sleep_time_s)

  return question,final_response

In [22]:
answer_chain = ( qna_prompt_template | llm | StrOutputParser() )

# **User Action Required**

Fill in your Question!

In [23]:
qn = 'How can I see the northern lights in Finland?'

**Sleep time (seconds) is added due to Groq Rate Limit :(**

In [24]:
qn, final_response = get_answer(qn,decomp_chain,custom_retriever,answer_chain, 60)

Main Question: How can I see the northern lights in Finland?

Sub Question: What are the best locations and time periods to see the northern lights in Finland?
The best locations to see the northern lights in Finland are in the north, particularly in Finnish Lapland [1, 2, 3]. This is where the northern lights season lasts for months, starting in September and continuing until spring [1, 2]. If the sky is cloudless, the northern lights statistically appear in Lapland almost every other night [2]. While they can occur year-round, the nights need to be dark enough for us to see the auroras, making autumn, winter, and spring the best times [1]. For instance, frozen lakes in these regions provide great open views to catch a glimpse of the Aurora Borealis [4]. Specifically, the best time to see the "ruska" in Northern Finland, which is the beginning of the northern lights season, is typically in September and October [3].

Citations:
[1]: 'Head north for a once-in-a-lifetime show: Northern 